# Performance

This section evaluates the performance of an airplane based on the given parameters. Specifically, the takeoff, landing, and climb gradient are estimated using the refined methods described in [performance section](../performance/intro.md). For takeoff and landing, only the total ground roll is computed since the requirement constraints the ground roll and not the total takeoff/landing distance. Note that cruise speed requirement is evaluated by comparing the available and required cruise power. Below code block defines functions for computing these performance metrics:

In [1]:
import nbimporter
import numpy as np
from parameters import AircraftParameters
from mtow import compute_takeoff_weight

def compute_takeoff_groundroll(parameters: AircraftParameters):
    """
        Function to compute takeoff ground roll
    """

    # Parameters
    S = parameters.S
    A = parameters.A
    e = parameters.e_takeoff
    b = parameters.b
    W0 = parameters.MTOW
    mu = parameters.mu
    h = parameters.Lm # wing height above ground
    rho = parameters.rho_sea_level
    CD0 = parameters.CD0_takeoff
    CLmax_TO = parameters.CLmax_takeoff
    P_takeoff = parameters.P_takeoff # total takeoff power, hp
    prop_eff = 0.8
    install_loss_factor = parameters.install_loss_factor

    # Computed
    W = parameters.W1_W0 * W0
    CL = 0.1 # Raymer recommendation
    wing_loading = W/S
    Vto = 1.1 * ( wing_loading * 2 / rho / CLmax_TO )**0.5 # ft/s

    T = 550 * P_takeoff * prop_eff / 0.707 / Vto # T at avg V
    T = install_loss_factor * T # installation loss
    T_avg = T # lbs

    # K in drag model, accounting for ground effect
    K = 1/np.pi/A/e
    Keff = K * ( 33 * (h/b)**1.5 / ( 1 + 33 * (h/b)**1.5 ) ) # eq 12.60, Raymer

    # Ground roll calculation
    KA = rho / 2 / wing_loading * (mu*CL - CD0 - Keff*CL**2)
    KT = (T_avg/W) - mu
    Sg = np.log( 1 + KA/KT*Vto**2 ) / 2 / parameters.g / KA # ft
    Sr = Vto * parameters.t_rotation # ft
    
    return Sg + Sr # ft

def compute_landing_groundroll(parameters: AircraftParameters):
    """
        Function to compute landing ground roll
    """

    # Parameters
    S = parameters.S
    A = parameters.A
    e = parameters.e_landing
    b = parameters.b
    W0 = parameters.MTOW
    mu = parameters.mu_brakes
    h = parameters.Lm # wing height above ground
    rho = parameters.rho_sea_level
    CD0 = parameters.CD0_landing
    CLmax_L = parameters.CLmax_landing

    # Computed
    W = parameters.W1_W0 * parameters.W2_W1 * W0 # assuming plane has to land just after climbing
    CL = 0.1 # Raymer recommendation
    wing_loading = W/S
    Vtd = 1.15 * ( wing_loading * 2 / rho / CLmax_L )**0.5 # ft/s

    # K in drag model, accounting for ground effect
    K = 1/np.pi/A/e
    Keff = K * ( 33 * (h/b)**1.5 / ( 1 + 33 * (h/b)**1.5 ) ) # eq 12.60, Raymer

    # braking distance distance
    KT = - mu # zero thrust
    KA = rho / 2 / wing_loading / S * (mu*CL - CD0 - Keff*CL**2)
    Sb = np.log( KT / (KT + KA*Vtd**2) ) / 2 / parameters.g / KA
    Sfr = Vtd * parameters.t_free_roll # ft
    
    return Sb + Sfr # ft

def compute_climb_gradient(parameters: AircraftParameters):
    """
        Function to compute climb gradient. Note that only the OEI requirement 
        is used since it the driving climb gradient constraint
    """

    # Parameters
    A = parameters.A
    S = parameters.S
    e = parameters.e
    prop_eff = parameters.prop_eff_cruise
    CLmax = parameters.CLmax
    CD0 = parameters.CD0
    W0 = parameters.MTOW
    P_takeoff = parameters.P_takeoff
    prop_area = parameters.prop_area

    # Calculations
    rho_5000 = 0.00204834 # slugs/cu ft, at 5000 ft
    sigma = rho_5000 / parameters.rho_sea_level
    CL = CLmax - 0.2 # 0.2 is safety margin
    K = 1/np.pi/A/e
    CD0_prop = 0.1 * sigma * prop_area / S
    CD0 += CD0_prop
    L_by_D = CL / (CD0 + K*CL**2)
    W = parameters.W2_W1 * parameters.W1_W0 * W0
    P_cruise = P_takeoff/2 * (sigma - (1-sigma)/7.75) # max power at cruising altitude

    CGR = 18.97 * prop_eff * (sigma*CL)**0.5 / (W/P_cruise) / (W/S)**0.5 - 1/L_by_D

    return CGR

Below code block shows how to use the above defined functions and evaluate performance of an airplane:

In [2]:
# Parameters
A = 8
wing_loading = 40 # lbs/sq ft
power_loading = 9.25 # lbs/hp

# Compute MTOW
MTOW, final_parameters = compute_takeoff_weight(wing_loading, power_loading, A)
final_parameters.MTOW = MTOW

nb = final_parameters.num_blades
blade_loading = final_parameters.blade_loading

final_parameters.prop_dia = ( 4 * final_parameters.P_takeoff / final_parameters.num_engines /  final_parameters.PI / final_parameters.num_blades / final_parameters.blade_loading)**0.5 # ft
final_parameters.prop_area =  final_parameters.PI * final_parameters.prop_dia**2 / 4

# Evaluate performance
tgr = compute_takeoff_groundroll(final_parameters)
lgr = compute_landing_groundroll(final_parameters)
cgr_oei = compute_climb_gradient(final_parameters)

print(f"Takeoff ground roll: {tgr:.0f} ft")
print(f"Landing ground roll: {lgr:.0f} ft")
print(f"CGR in OEI condition: {cgr_oei:.4f}")

Takeoff ground roll: 1107 ft
Landing ground roll: 754 ft
CGR in OEI condition: 0.0160


This concludes the performance evaluation and code setup section. The next section performs trade studies using the functions defined in this and previous sections.